In [4]:
# ====== 1. 환경 설정 및 라이브러리 임포트 ======
import os
import ast
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv

# 출력 옵션 설정
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

# .env 파일 로드 (환경변수)
load_dotenv()

# 데이터 경로 설정
path = '../../steam_data/new_steam/purchased_games.csv'


In [ ]:
import json
import pandas as pd

path = "../../steam_data/new_steam/purchased_games.csv"
df = pd.read_csv(path)

# 1) library가 JSON 문자열인지 확인
print(df.columns)
print(df["library"].head(2).to_list())

# 2) JSON 파싱 (에러나는 행은 None)
def safe_loads(x):
    if pd.isna(x):
        return None
    if isinstance(x, (list, dict)):
        return x
    x = str(x).strip()
    if not x:
        return None
    try:
        return json.loads(x)
    except Exception:
        return None

df["library_obj"] = df["library"].apply(safe_loads)

# 3) library_obj가 리스트(=게임 목록)면 explode해서 user-item으로 펼치기
# (dict인 경우도 있는데, 그때는 dict 안에 games/items 같은 키가 있을 수 있어서 아래에서 처리)
tmp = df[["playerid", "library_obj"]].copy()

# dict면 내부 키를 추정해서 리스트로 바꿈
def dict_to_list(obj):
    if obj is None:
        return None
    if isinstance(obj, list):
        return obj
    if isinstance(obj, dict):
        # 흔한 케이스 후보들
        for k in ["games", "items", "library", "owned_games", "data"]:
            if k in obj and isinstance(obj[k], list):
                return obj[k]
        # 마지막 수단: dict 값들 중 list 찾기
        for v in obj.values():
            if isinstance(v, list):
                return v
    return None

tmp["games_list"] = tmp["library_obj"].apply(dict_to_list)
tmp = tmp.dropna(subset=["games_list"])
tmp = tmp.explode("games_list", ignore_index=True)

# 4) games_list의 각 원소가 dict라면 columns로 펼치기
games_df = pd.json_normalize(tmp["games_list"])
out = pd.concat([tmp[["playerid"]].reset_index(drop=True), games_df], axis=1)

print(out.columns)
print(out.head(5))

# 5) 표준 컬럼으로 정리 (키 이름은 데이터셋에 따라 다를 수 있어 후보들을 같이 처리)
def pick_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

col_app = pick_col(out, ["appid", "app_id", "gameid", "game_id", "id"])
col_play_forever = pick_col(out, ["playtime_forever", "playtime_total", "playtime", "hours", "playtime_minutes"])
col_play_2w = pick_col(out, ["playtime_2weeks", "playtime_last_two_weeks", "playtime_2_weeks"])
col_last = pick_col(out, ["time_last_played", "last_played", "rtime_last_played", "last_played_time"])

print("app:", col_app)
print("play_forever:", col_play_forever)
print("play_2weeks:", col_play_2w)
print("last_played:", col_last)

# 6) 최종 interaction 테이블
cols = ["playerid"]
if col_app: cols.append(col_app)
if col_play_forever: cols.append(col_play_forever)
if col_play_2w: cols.append(col_play_2w)
if col_last: cols.append(col_last)

interactions = out[cols].copy()
interactions = interactions.rename(columns={"playerid": "user_id", col_app: "app_id"})
print(interactions.head())


Index(['playerid', 'library'], dtype='object')
['[60, 1670, 3830, 1600, 2900, 2910, 2920, 4800, 4000, 6200, 6210, 6300, 6900, 6850, 6860, 6870, 1610, 1690, 6980, 7010, 1700, 6310, 6510, 4540, 475150, 4550, 2310, 7670, 409710, 6270, 8600, 44630, 2590, 2500, 11200, 8100, 15100, 20, 50, 70, 130, 220, 280, 320, 360, 15130, 11230, 12750, 11250, 16710, 12320, 15300, 13630, 13620, 4420, 4470, 11260, 15700, 15710, 20900, 13540, 15000, 15120, 19830, 19840, 22000, 18300, 12200, 16730, 19980, 9480, 12770, 17480, 17410, 15620, 20570, 56400, 19000, 11590, 11560, 21800, 23310, 21900, 29800, 11340, 22200, 21640, 1930, 16720, 3590, 8170, 12790, 10150, 22330, 12460, 12470, 12440, 12450, 9870, 34800, 17390, 17440, 24720, 35700, 34900, 10100, 10130, 10110, 21670, 21680, 18480, 25890, 11370, 36000, 6830, 6840, 29180, 38400, 38410, 38420, 11390, 37400, 37420, 35000, 6010, 6040, 32310, 32340, 25980, 34600, 6800, 6810, 35070, 40700, 24800, 24840, 32410, 34820, 34810, 41500, 12330, 33570, 33580, 17450, 47810,

In [8]:
import pandas as pd
from pathlib import Path

BASE = Path("../../steam_data/new_steam")  # 너 경로에 맞게

files = {
    "players": BASE/"players.csv",
    "games": BASE/"games.csv",
    "achievements": BASE/"achievements.csv",
    "history": BASE/"history.csv",
    "reviews": BASE/"reviews.csv",
    "purchased": BASE/"purchased_games.csv",
    "private_ids": BASE/"private_steamids.csv",
    "friends": BASE/"friends.csv",
}

dfs = {k: pd.read_csv(v) for k, v in files.items()}

# 컬럼 확인
for k, df in dfs.items():
    print(k, df.shape, list(df.columns))

def to_dt(series):
    # unix seconds, unix ms, ISO string 다 커버
    s = series.copy()
    # 숫자면 unix 가능성
    if pd.api.types.is_numeric_dtype(s):
        # ms인지 sec인지 대충 판별
        if s.dropna().astype("int64").median() > 10**12:
            return pd.to_datetime(s, unit="ms", errors="coerce")
        else:
            return pd.to_datetime(s, unit="s", errors="coerce")
    # 문자열이면 ISO/기타 포맷
    return pd.to_datetime(s, errors="coerce")

# history 최신
if "date_acquired" in dfs["history"].columns:
    dt = to_dt(dfs["history"]["date_acquired"])
    print("history max:", dt.max(), "min:", dt.min())

# reviews 최신
if "posted" in dfs["reviews"].columns:
    dt = to_dt(dfs["reviews"]["posted"])
    print("reviews max:", dt.max(), "min:", dt.min())

# players 최신
if "created" in dfs["players"].columns:
    dt = to_dt(dfs["players"]["created"])
    print("players.created max:", dt.max(), "min:", dt.min())


players (424683, 3) ['playerid', 'country', 'created']
games (98248, 7) ['gameid', 'title', 'developers', 'publishers', 'genres', 'supported_languages', 'release_date']
achievements (1939027, 4) ['achievementid', 'gameid', 'title', 'description']
history (10693879, 3) ['playerid', 'achievementid', 'date_acquired']
reviews (1204534, 8) ['reviewid', 'playerid', 'gameid', 'review', 'helpful', 'funny', 'awards', 'posted']
purchased (102548, 2) ['playerid', 'library']
private_ids (227963, 1) ['playerid']
friends (424683, 2) ['playerid', 'friends']
history max: 2024-12-15 19:42:05 min: 2008-09-13 01:37:54
reviews max: 2025-01-09 00:00:00 min: 2010-10-15 00:00:00
players.created max: 2025-01-07 19:57:59 min: 2003-09-11 08:52:38


## 결과 : 최신 데이터셋 2025-01-12일 까지 cover
- 이후최신 데이터는 api를 통한 커버를 통해서 가능하다 